In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
inspector = inspect(engine)
inspector.get_columns('station')

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
precip_df = pd.read_sql('measurement', engine, index_col = 'id')
precip.tail()

# Exploratory Climate Analysis

In [ ]:
import datetime as dt

In [ ]:
# Calculate the date 1 year ago from the last data point in the database
year_ago = dt.date(2017, 8, 23) - dt.timedelta(days = 365)
year_ago

In [ ]:
# Perform a query to retrieve the data and precipitation scores
results = session.query( Measurement.date, Measurement.prcp).\
filter(Measurement.date > '2016-08-23').all()

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
precip = pd.DataFrame(results, columns = [ 'date', 'Precipitation'])
precip.tail()

In [ ]:
# Sort the dataframe by date
date_sort = precip.groupby(['date']).sum()
date_sort

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Use Pandas Plotting with Matplotlib to plot the data
precip.plot.bar(title = "Precip in HI over 12 months", figsize =(20, 10), width = 4)
plt.xlabel("Date")
plt.ylabel("Precip (in)")

plt.tight_layout()

![precipitation](Images/precipitation.png)

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precip.describe()

![describe](Images/describe.png)

In [ ]:
# Design a query to show how many stations are available in this dataset?
session.query(func.count(Station.id)).all()

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
results1 = session.query(Measurement.station, Measurement.tobs).\
filter(Measurement.tobs != 'NaN').all()
station_df = pd.DataFrame(results1, columns=['Station', 'TOBS'])
grouped_st = station_df.groupby(['Station']).count()
grouped_st.sort_values('TOBS', ascending = False)

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
filter(Measurement.station == 'USC00519281').all()

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
results2 = session.query(Measurement.station, Measurement.tobs).\
filter(Measurement.station == 'USC00519281').filter(Measurement.date > '2016-08-23').all()
year_temp = pd.DataFrame(results2, columns = ['Station', 'TOBS'])
year_temp.hist('TOBS')
plt.title("Time Observed Biased Temp Histogram")
plt.ylabel('Frequency')

![precipitation](Images/station-histogram.png)

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
results3 = calc_temps('2017-03-25', '2017-03-29')
trip_temps = pd.DataFrame(results3, columns = ['Min Temp', 'Avg Temp', 'Max Temp'])
trip_temps

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
ptp = trip_temps['Max Temp'] - trip_temps['Min Temp']
trip_temps['Avg Temp'].plot.bar(yerr = ptp/2)
plt.ylabel("Temperature (F)")
plt.xlabel("Trip Dates AVG")
plt.title('Average Temperature Over Planned Trip Dates')
plt.tight_layout()

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
results4 = session.query(Measurement.station, Measurement.prcp, Station.name, Station.latitude, Station.longitude, Station.elevation).\
filter(Measurement.date > '2017-03-24').filter(Measurement.date<'2017-03-30').filter(Measurement.prcp != 'NaN').filter(Measurement.station == Station.station).group_by(Measurement.station).all()
stations = pd.DataFrame(results4, columns = ['ID', 'Precip Total', 'Name', 'Latitude', 'Longitude', 'Elevation'])
stations.sort_values(['Precip Total'], ascending = False )



## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)
, Station.name, Station.latitude, Station.longitude, Station.elevation, 'Name', 'Latitude', 'Longitude', 'Elevation'
, 'Name', 'Latitude', 'Longitude', 'Elevation'
def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
